In [131]:
import random
import pandas as pd

def hamming_distance(string1, string2):
    """Calculates the Hamming distance between two strings"""
    distance = 0
    for i in range(len(string1) - 1):
        if string1[i] != string2[i]:
            distance += 1
    return distance

def distance_matrix(strings):
    """Calculate the Hamming distances between all strings"""
    matrix = [[0] * len(strings) for _ in strings]
    for i in range(len(strings)):
        for j in range(len(strings)):
            dist = hamming_distance(strings[i], strings[j])
            matrix[i][j] = dist
    return matrix

def avg_dist_within_group(element, element_list, dist_matrix):
    """Find the average Hamming distance between a given element and the main group of elements"""
    diameter = -float("inf")
    total_dist = 0
    for i in element_list:
        total_dist += dist_matrix[element][i]
        if dist_matrix[element][i] > diameter:
            diameter = dist_matrix[element][i]
    if len(element_list) > 1:
        avg_dist = total_dist / (len(element_list) - 1)
    else:
        avg_dist = 0
    return avg_dist

def avg_dist_across_group(element, splinter_list, dist_matrix):
    """Find the average Hamming distance between a given element and the split cluster"""
    if len(splinter_list) == 0:
        return 0
    total_dist = 0
    for j in splinter_list:
        total_dist += dist_matrix[element][j]
    avg_dist = total_dist / len(splinter_list)
    return avg_dist

def splinter(main_list, splinter_group, dist_matrix):
    """Find the string location to split the main list at"""
    max_value = -float("inf")
    farthest_index = None
    for string in main_list:
        within_dist = avg_dist_within_group(string, main_list, dist_matrix)
        across_dist = avg_dist_across_group(string, splinter_group, dist_matrix)
        diff = within_dist - across_dist
        if diff > max_value:
            max_value = diff
            farthest_index = string
    if max_value > 0:
        return farthest_index, True
    else:
        return False, False

def split(element_list, dist_matrix):
    """Split a main list into two clusters"""
    main_list = element_list
    splinter_group = []
    farthest_index, flag = splinter(main_list, splinter_group, dist_matrix)
    while flag:
        main_list.remove(farthest_index)
        splinter_group.append(farthest_index)
        farthest_index, flag = splinter(element_list, splinter_group, dist_matrix)

    return main_list, splinter_group

def max_cluster_distance(clusters, dist_matrix):
    """Find the largest distance difference within a cluster"""
    max_cluster_index = None
    max_cluster_dist = -float("inf")
    index = 0
    for elements in clusters:
        for i in elements:
            for j in elements:
                if dist_matrix[i][j] > max_cluster_dist:
                    max_cluster_dist = dist_matrix[i][j]
                    max_cluster_index = index

        index += 1

    if max_cluster_dist <= 0:
        return -1

    return max_cluster_index

In [150]:
import copy 
from collections import defaultdict

path_to_sample_file = "descendants_k=8_g=12_m=8_s=2.txt"
sample_number = 100


for random_seed, label in enumerate(['a', 'b', 'c', 'd']):
    with open(path_to_sample_file, "r") as f:
        strings = f.read()[:-1].split("\n")
        random.seed(random_seed)
        random.shuffle(strings)
        input_strings = random.sample(strings, sample_number)
        dist_matrix = distance_matrix(input_strings)

        # We use Pandas to append string values to the distance matrix
        dist_matrix = pd.DataFrame(dist_matrix, index=input_strings, columns=input_strings)

        all_levels = defaultdict(lambda:defaultdict(lambda:[]))
        current_clusters = ([input_strings])
        level = 1
        index = 0
        while index != -1 and level < 30:
            print(level)
            #print(level, current_clusters, len(current_clusters))
            # Storing cluster values in the line below, need to convert to defaultdict
            all_levels[level] = copy.deepcopy(current_clusters)

            main_list, split_cluster = split(current_clusters[index], dist_matrix)

            del current_clusters[index]
            current_clusters.append(main_list)
            current_clusters.append(split_cluster)
            index = max_cluster_distance(current_clusters, dist_matrix)
            level += 1
        #all_levels[level] = current_clusters[index]

    with open('descendants_k=8_g=12_m=8_s=2.100_sampled.{}_clusters.txt'.format(label), 'w') as f:
        for cluster_num, clusters in all_levels.items():
            f.write(">{}\n".format(cluster_num))
            for c in clusters:
                f.write('{}\n'.format(' '.join(c)))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [141]:
import numpy as np

for level_number, clusters  in all_levels.items():
    print('\n~~~', level_number)
    pos_counts = defaultdict(lambda:defaultdict(lambda:0))
    for c in clusters:
        print('\t\n\n', len(c))
        for seq in c:
            for i, nuc in enumerate(seq):
                pos_counts[i][nuc] += 1
            
        reconstructed = ''
        for pos in sorted(pos_counts.keys()):
            counts = pos_counts.get(pos)
            max_count = max(counts.values())
            for n in ['A', 'C', 'T', 'G']:
                if max_count == counts.get(n):
                    reconstructed += n
                    break

        # hamming distance between reconstructed string and each original cluster member
        print(reconstructed)
        all_hamming_distances = []
        for seq in c:
            all_hamming_distances.append(hamming_distance(seq, reconstructed))
        print(np.mean(all_hamming_distances))
        print(np.std(all_hamming_distances))

    if level_number > 10:
        break


~~~ 1
	

 1300
ACCAGAAGTCCAAACAGCACGATATCGGGTAAGAGCTGTCCACGATTGGTTATTTTCTTATACCATGCTCGGAACCTGTAGGGTGGTCGATGCGACCGAG
62.12230769230769
4.127166384683795

~~~ 2
	

 962
CGAACCAGTCCAAACAGAACGATATCGGTCAAGAGATTGCCAGGTTTCGTTATTTTTATAAACCATGATCGCAACCTGTTGTGTGGTCGATGCGTCCGAG
60.45530145530145
4.108023937121163
	

 338
ACCAGAAGTCCAAACAGCACGATATCGGGTAAGAGCTGTCCACGATTGGTTATTTTCTTATACCATGCTCGGAACCTGTAGGGTGGTCGATGCGACCGAG
62.26331360946745
4.020496704851294

~~~ 3
	

 338
ACCAGAAGTGGCTATCGCGCTAAAGCTGGTCCCTACGGTCAGCGGGTGGCCATTTACTCGTGAAAACCATAGTCCGTGAAAGCTGGGCAATTTTAGTTGG
45.84023668639053
6.243427972147019
	

 644
ACCACAAGAAAAAACACCACGATACCTAGCAGCAACTGTCCACGTTAGGAAGTTTTCTTATAACAACCTAGGGACATGAAGGGTGGGCGATGAGAGCTTG
60.38354037267081
5.378567071168052
	

 318
ACCAGAAGTCCAAACAGCACGATATCGGGTAAGAGCTGTCCACGATTGGTTATTTTCTTATACCATGCTCGGAACCTGTAGGGTGGTCGATGCGACCGAG
61.12893081761006
4.146573123456256

~~~ 4
	

 338
ACCAGAAGTGGCTATCGCGCTAAAGCTGGTCCCTACGGTCAGCGGGTGGCCATTTACTCGTGAAAACCATAGTCCGTGAAAGCTGGGCAATTT